<a href="https://colab.research.google.com/github/Lazuardis/DESinPython/blob/main/Coffee_%26_Pizza_Eatery_Chapter_1_Basic_Discrete_Event_Simulation_Model_in_SimPy_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Coffee & Pizza Eatery** 🍕🥤




Imagine you run a coffee & pizza eatery just around the corner of city main avenue. You want to observe how the eatery's daily operational take shape as you been told that, for some ocassion, the waiting time is too long, or some customer complaining why the tables are always occupied.

You then managed to gather some data from direct observation. Some data like processing time, customer interarrival time, until customer preference have been collected. And now using discrete event simulation, you try to actually understand how the eatery work, in a more quantitatively representative model.


In [1]:
pip install simpy

In [2]:
# import the corresponding libraries

import simpy
import pandas as pd
import numpy as np
import random

### **Chapter 1: Basic DES Model Building**


#### Simulation Data as Input Parameter:


---


When running a simulation model, it's important to have input parameters that define and influence the system. Different inputs will result in distinct outputs, and understanding their relationship is crucial.

In some cases, input parameters are environmental factors, such as the frequency of customer arrivals at a restaurant or the typical dining duration. However, there are also instances where input parameters are within our control or observable, allowing us to determine how to set them. For instance, the number of staff or the quantity of tables can be adjusted to achieve specific outcomes.

Code below will demonstrate how we translate observed data into python's variable. However this code will not define ALL of required input parameter within this one cell. We will explain why later!

In [3]:
# define the neccessary data to be involved in the simulation
# NOTE: time units interpreted across the notebook are in MINUTES

inter_arrival_time = random.expovariate(1/5) #customer arriving in every 5 minutes

processing_time = {
  "till_process": random.uniform(1,3), #till process duration is uniformly distributed from a minute through 3 minutes per customer
  "coffee_process": random.gauss(1,0.5), #coffee making process duration is normally distributed averaging one minute with 0.5 min as standard deviation
  "pizza_process": random.gauss(5,1),
  "dining_in": random.gauss(15,5)
}

### Building the model through sequencing process into **function**


---

Simpy library package model discrete event system quite literally. So, it is better to understand the system indeed as sequence of discrete event, which we illustrate them as human-driven activities.

Simpy strenght relies on the way it could model how time elapsed due to:

1.   Undergoing process (with or without resource)
2.   Waiting for a resource being requested

Hence, we structure the model as distinct blocks of events, each represented by a Python function. These functions are responsible for simulating the passage of time as the associated event unfolds.


**Event 1: Customer Arrival**

Customer arrival is modeled based on the distribution value of inter arrival rate defined earlier.

Here, we also inject an input parameter called *customer_type* to denote how many people are there in each set of customer arriving into the eatery.

We have 40% of them are single individual, 30% are couple, 20% are group of three, and 10% are group of four.

**Important Note**: As we said earlier, not all input parameter will be predefined in designated cell like above (externally from the *def function* code). Sometimes, it is better this way as predefining input parameter will need us to include them as *function* input argument. The thing is some of input parameter will be used only by specific function, so it will be more convenient and neat to define them as *local variable* to associated funtion.


In [4]:
def customer_arrival(env, inter_arrival_time):
  global customer
  global customer_served
  customer = 0 #represent the customer ID
  while True: #while the simulation is still in condition to be run
    yield env.timeout(inter_arrival_time)
    customer += 1 #customer ID added
    customer_type = random.choices([1,2,3,4], [0.4,0.3,0.2,0.1])[0]
    print(f"customer {customer} arrives at {env.now:7.4f}")

    next_process = till_activity(env, processing_time, customer, customer_type)
    env.process(next_process) #next process is integrated within this function


**Another Coding Important Note:**

till process as the next process after customer arrival, is called within the customer_arrival function. It is important to structure the code this way as till process **could only happen only if customer arrival event is done.**

This makes sure that the two events/process is conducted in sequential fashion. And this too, apply for the rest of the remaining later process/event

**Event 2: Till Activity**

After till activity, customer will randomly proceed to order between:


*   Pizza only
*   Coffee only
*   Coffee and Pizza




In [18]:
def till_activity(env, processing_time, customer, customer_type):
  with staff.request() as till_request: #requesting staff to service at the till
    yield till_request #waiting until the staff available
    yield env.timeout(processing_time["till_process"]) #elapsed time of till activity, staff resource is automatically released after it
    print(f"till complete at {env.now:7.4f} for customer {customer}")

  order_type = random.randint(1,3) #random assignment for customer ordering type
  dining_in = random.randint(0,1) #random assignment for whether customer intend to dine in or take away

  order_coffee = coffee_activity(env, processing_time, customer, customer_type, dining_in)
  order_pizza = pizza_activity(env, processing_time, customer, customer_type, dining_in)
  order_all = coffee_pizza_activity(env, processing_time, customer, customer_type, dining_in)

  if order_type == 1: # if customer order type is only ordering coffee, then proceed to order coffee process
    env.process(order_coffee)
  elif order_type == 2: # same logic with above
    env.process(order_pizza)
  else: env.process(order_all) # if neither only coffee nor only pizza, then they must order both!




**Event 3.1: Coffee Activity**

Only proceed to this sub-event if customer order coffee only

In [19]:
def coffee_activity(env, processing_time, customer, customer_type, dining_in):
  global customer_served
  with staff.request() as coffee_request:
    yield coffee_request
    yield env.timeout(processing_time["coffee_process"])
    print(f"order complete at {env.now:7.4f} for customer {customer}")

  dining_process = dining_activity(env, processing_time, customer, customer_type)
  if dining_in == 1:
    env.process(dining_process) #if customer intend to dine in, proceed to dine in process
  else:
    customer_served += 1 #customer is successfully served
    print(f"Customer {customer} leaves at {env.now:7.4f}") #if customer intend to take away, they leave


**Event 3.2: Pizza Activity**

Only proceed to this sub-event if customer order pizza only

In [20]:
def pizza_activity(env, processing_time, customer, customer_type, dining_in):
  global customer_served
  with staff.request() as pizza_request:
    yield pizza_request
    yield env.timeout(processing_time["pizza_process"])
    print(f"order complete at {env.now:7.4f} for customer {customer}")

  dining_process = dining_activity(env, processing_time, customer, customer_type)
  if dining_in == 1:
    env.process(dining_process) #if customer intend to dine in, proceed to dine in process
  else:
    customer_served += 1
    print(f"Customer {customer} leaves at {env.now:7.4f}") #if customer intend to take away, they leave

**Event 3.3: Coffee & Pizza Activity**

Only proceed to this sub-event if customer order coffee & pizza

In [21]:
def coffee_pizza_activity(env, processing_time, customer, customer_type, dining_in):
  global customer_served
  with staff.request() as pizza_request:
    yield pizza_request
    yield env.timeout(processing_time["pizza_process"] + processing_time["coffee_process"])
    print(f"order complete at {env.now:7.4f} for customer {customer}")

  dining_process = dining_activity(env, processing_time, customer, customer_type)
  if dining_in == 1:
    env.process(dining_process) #if customer intend to dine in, proceed to dine in process
  else:
    customer_served += 1
    print(f"Customer {customer} leaves at {env.now:7.4f}") #if customer intend to take away, they leave

**Event 4: Dining Activity**

Customer only proceed to this activity only if *customer_type* == 1, or is intending to dine in.

However, even if they intend to dine in, customer will decide to change to take away if they found there is no seat available for them to dine in. Usually it takes them around 10 second to confirm whether there is available table or not.





In [22]:
def dining_activity(env, processing_time, customer, customer_type):
  global customer_served
  if customer_type <= 2:
    with two_seater.request() as twoseater_request:
      decision = yield twoseater_request | env.timeout(10/60) # the decision is whether there is available two seater or not

      if twoseater_request in decision:
        yield env.timeout(processing_time["dining_in"]) # customer found two seater and dining in
        customer_served += 1
        print(f"Dining in complete at {env.now:7.4f} for customer {customer}")
        print(f"Customer {customer} leaves at {env.now:7.4f}")
      else:
        print(f"Customer {customer} leaves at {env.now:7.4f}") # after 10 seconds check, customer found no seat available, hence take away
        customer_served += 1

  else:
    with four_seater.request() as fourseater_request:
      decision = yield fourseater_request | env.timeout(2) # same exact scenario for group of three or four looking for four seater

      if fourseater_request in decision:
        yield env.timeout(processing_time["dining_in"])
        print(f"Dining in complete at {env.now:7.4f} for customer {customer}")
        print(f"Customer {customer} leaves at {env.now:7.4f}")
        customer_served += 1
      else:
        print(f"Customer {customer} leaves at {env.now:7.4f}")
        customer_served += 1



### Setting the main model run


---

the main model code has it needs to set up all the underlying command required to activate all of developed functions above.

this means creating variable for simpy environment, setting the input parameter of resource (followed by its capacity), and summon the main process that binds every other succeeding process.

In our case, customer_arrival is the main or initial process that will be followed by till activity, which later followed by preparing the coffee and/or pizza, and optional dining in. As we have made sure all of these activities will be conducted sequentially, summoning only *customer_arrival* process is enough, through `env.process(customer_arrival(env, inter_arrival_time)) `




In [10]:
random.seed(100) #random seed to preserve same random number generated

env = simpy.Environment() #create the essential simpy environment

staff = simpy.Resource(env, capacity = 2) #staff
two_seater = simpy.Resource(env, capacity = 4) #two seater for one or couple customer
four_seater = simpy.Resource(env, capacity = 1) #four seater for three or four group of customer

customer = 0 #set the initial customer id starting from 0
customer_served = 0 #number of customer served during the start of simulation is zero
env.process(customer_arrival(env, inter_arrival_time))


env.run(until=60*4) # run the simulation for 3 hours
print('\n')
print(f"TOTAL COMPLETE CUSTOMER:{customer_served}")
print(f"Customer in System:{customer - customer_served}")


customer 1 arrives at  3.4421
till complete at  4.6837 for customer 1
customer 2 arrives at  6.8841
till complete at  8.1258 for customer 2
order complete at  8.9441 for customer 1
Customer 1 leaves at  8.9441
customer 3 arrives at 10.3262
till complete at 11.5678 for customer 3
order complete at 13.2129 for customer 3
Customer 3 leaves at 13.2129
customer 4 arrives at 13.7683
order complete at 14.0312 for customer 2
till complete at 15.0099 for customer 4
customer 5 arrives at 17.2103
till complete at 18.4520 for customer 5
order complete at 19.2703 for customer 4
customer 6 arrives at 20.6524
till complete at 21.8940 for customer 6
customer 7 arrives at 24.0945
order complete at 24.3574 for customer 5
Customer 5 leaves at 24.3574
till complete at 25.5991 for customer 7
order complete at 26.1544 for customer 6
Customer 6 leaves at 26.1544
order complete at 27.2442 for customer 7
Customer 7 leaves at 27.2442
customer 8 arrives at 27.5365
till complete at 28.7782 for customer 8
customer

In [23]:
random.seed(100) #random seed to preserve same random number generated

env = simpy.Environment() #create the essential simpy environment

staff = simpy.Resource(env, capacity = 2) #staff
two_seater = simpy.Resource(env, capacity = 4) #two seater for one or couple customer
four_seater = simpy.Resource(env, capacity = 1) #four seater for three or four group of customer

customer = 0 #set the initial customer id starting from 0
customer_served = 0 #number of customer served during the start of simulation is zero
env.process(customer_arrival(env, inter_arrival_time))


env.run(until=60*4) # run the simulation for 3 hours
print('\n')
print(f"TOTAL COMPLETE CUSTOMER:{customer_served}")
print(f"Customer in System:{customer - customer_served}")


customer 1 arrives at  3.4421
till complete at  4.6837 for customer 1
customer 2 arrives at  6.8841
till complete at  8.1258 for customer 2
order complete at  8.9441 for customer 1
Customer 1 leaves at  8.9441
customer 3 arrives at 10.3262
till complete at 11.5678 for customer 3
order complete at 13.2129 for customer 3
Customer 3 leaves at 13.2129
customer 4 arrives at 13.7683
order complete at 14.0312 for customer 2
till complete at 15.0099 for customer 4
customer 5 arrives at 17.2103
till complete at 18.4520 for customer 5
order complete at 19.2703 for customer 4
customer 6 arrives at 20.6524
till complete at 21.8940 for customer 6
customer 7 arrives at 24.0945
order complete at 24.3574 for customer 5
Customer 5 leaves at 24.3574
till complete at 25.5991 for customer 7
order complete at 26.1544 for customer 6
Customer 6 leaves at 26.1544
order complete at 27.2442 for customer 7
Customer 7 leaves at 27.2442
customer 8 arrives at 27.5365
till complete at 28.7782 for customer 8
customer

### **Modifying Stopping Criteria**

Now, you can run the simulation and observe the result!. Here, we still only showcase very basic output which are total customer that is finished their journey in the eatery. And we also print the output of customer in system, meaning how many customer still not finish by the time simulation time complete.

You can modify this simulation stopping criteria as for instance you want to observe how long does it take to service completely 100 customer. Below are the code that sees the modification in customer_arrival function and main model run function

In [ ]:
stop_criteria = env.event()

def customer_arrival(env, inter_arrival_time):
  global customer
  global customer_served
  customer = 0 #represent the customer ID
  customer_served

  while True:
    if customer_served >= 100:
      stop_criteria.succeed()
    else:
      yield env.timeout(inter_arrival_time)
      customer += 1 #customer ID added
      customer_type = random.choices([1,2,3,4], [0.4,0.3,0.2,0.1])[0]
      print(f"customer {customer} arrives at {env.now:7.4f}")
      next_process = till_activity(env, processing_time, customer, customer_type)
      env.process(next_process) #next process is integrated within this function

In [ ]:
try:
  random.seed(1000) #random seed to preserve same random number generated

  env = simpy.Environment() #create the essential simpy environment

  staff = simpy.Resource(env, capacity = 2) #staff
  two_seater = simpy.Resource(env, capacity = 4) #two seater for one or couple customer
  four_seater = simpy.Resource(env, capacity = 1) #four seater for three or four group of customer

  customer = 0 #set the initial customer id starting from 0
  customer_served = 0 #number of customer served during the start of simulation is zero
  env.process(customer_arrival(env, inter_arrival_time))

  env.run(until=stop_criteria) # now we don't need time as stopping criteria here

except Exception:
    pass

print('\n')
print(f"TOTAL COMPLETE CUSTOMER:{customer_served}")
print(f"Customer in System:{customer - customer_served}")

customer 1 arrives at  0.7872
customer 2 arrives at  1.5744
customer 3 arrives at  2.3616
till complete at  2.6970 for customer 1
customer 4 arrives at  3.1487
till complete at  3.4842 for customer 2
customer 5 arrives at  3.9359
till complete at  4.6069 for customer 3
customer 6 arrives at  4.7231
customer 7 arrives at  5.5103
customer 8 arrives at  6.2975
till complete at  6.5167 for customer 4
order complete at  6.9339 for customer 1
Customer 1 leaves at  6.9339
customer 9 arrives at  7.0847
customer 10 arrives at  7.8718
customer 11 arrives at  8.6590
till complete at  8.8437 for customer 5
customer 12 arrives at  9.4462
customer 13 arrives at 10.2334
customer 14 arrives at 11.0206
order complete at 11.0682 for customer 2
customer 15 arrives at 11.8078
customer 16 arrives at 12.5949
till complete at 12.9781 for customer 6
customer 17 arrives at 13.3821
order complete at 13.3952 for customer 3
Customer 3 leaves at 13.3952
customer 18 arrives at 14.1693
till complete at 14.8879 for c

**Note**: the code above is integrated within below pass exception code for I still have an issue with smoothly terminating the `env.run()`


```
try:
except Exception:
    pass
```

However, this should not be a problem and does do the trick. The resulting code is working as intended.
